In [14]:
import pandas as pd
import geopandas as gpd
import xgboost
import sys
from sklearn.preprocessing import LabelEncoder
import numpy as np
from math import * 
from tqdm import tqdm
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.cluster import DBSCAN, OPTICS
from xgboost import XGBClassifier


import json
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 250
plt.style.use('dark_background')

from utils import plot_cluster, load_list, save_list, load_data

In [9]:
X, Y = load_data(add_knn_mean=False,
                 add_knn_concat=False,
                 n_data_max=1000,
                 shuffle = True)


Loading data...


c:\Users\timot\MLcompetition\utils.py:47: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = train_df.drop("change_type", 1).reindex(idx)[:n_data_max]


In [31]:
X, Y = load_data(add_knn_mean=False,
                 add_knn_concat=False,
                 n_data_max=300000,
                 shuffle = True)


Loading data...


c:\Users\timot\MLcompetition\utils.py:47: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = train_df.drop("change_type", 1).reindex(idx)[:n_data_max]


In [34]:

rf = RandomForestClassifier(n_estimators=100,
                            max_depth=5,
                            min_samples_leaf=1,
                            min_samples_split=2,
                            )

xgb = XGBClassifier()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 22)
n_data_train = len(X_train)
n_step = 1

print('Fitting...')
for k in range(n_step):
        N1, N2 = k*n_data_train//n_step, (k+1)*n_data_train//n_step
        rf.fit(X_train[N1: N2] , Y_train[N1: N2] )
        Y_pred          = rf.predict(X_test)
        Y_pred_train    = rf.predict(X_train)
        score_train = f1_score(Y_train, Y_pred_train, average = 'micro')
        score_test = f1_score(Y_test, Y_pred, average = 'micro')
        print(f"Train score: {score_train}")
        print(f"Test score: {score_test}")
        print()

Fitting...
Train score: 0.6134833333333334
Test score: 0.6098166666666667



In [6]:
# skf = StratifiedKFold(n_splits=5)
# n_step = 8
# for train_index, test_index in skf.split(X, Y):
#     X_train = X[train_index]
#     Y_train = Y[train_index]
#     X_test = X[test_index]
#     Y_test = Y[test_index]
#     n_data_train = len(X_train)
    
#     for k in range(n_step-1):
#         rf.fit(X_train[k*n_data_train//n_step: (k+1)*n_data_train//n_step])
#         Y_pred = rf.predict(X_test)
#         score = f1_score(Y_test, Y_pred, )

(1000,)